In [55]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np
import json
import re


In [3]:
beautify = lambda req: [[bs(r,"html.parser")] for r in req]
getItems = lambda req: [[r[0].find_all("a",{"class":"item_row_flex"})] for r in req] 
itemsToList = lambda req: [[r.findAll("div",{"class":"li-title"}) \
                             ,r.findAll("p",{"class":"list_price"}) \
                             ,r["href"]
                             ,r.findAll("img",{"class":"item_image"})\
                           ] for r in req]

filterL = lambda inp: list(map(lambda x: {"Name":x[0][0].text,"Price":x[1][0].text,"Link":x[2],"Image":x[3]},inp))

flatten = lambda f : [item for sublist in f for item in sublist]


In [52]:
reqList = []
for i in range(0,50):
    guitarReq = requests.get("https://www.tori.fi/koko_suomi/musiikki_ja_soittimet?ca=18&cg=4070&w=3&o="+str(i))
    reqList.append(guitarReq.text)


KeyboardInterrupt: 

In [10]:
    #guitarReq = requests.get("https://www.tori.fi/koko_suomi?q=&cg=4070&w=3&st=s&st=k&st=u&st=h&st=g&c=4072&ps=&pe=5&ca=18&l=0&md=th")

#keyReq = requests.get("https://www.tori.fi/koko_suomi?q=&cg=4070&w=3&st=s&st=k&st=u&st=h&st=g&c=4074&ps=&pe=5&ca=18&l=0&md=th")
#otherReq = requests.get("https://www.tori.fi/koko_suomi?q=&cg=4070&w=3&st=s&st=k&st=u&st=h&st=g&c=4079&ps=&pe=5&ca=18&l=0&md=th")
#reqList = [guitarReq.text,keyReq.text,otherReq.text]

soups = beautify(reqList)
tl = getItems(soups)
itemsList = itemsToList(flatten(flatten(tl)))

In [12]:
df = pd.DataFrame(filterL(itemsList))
df["Image"] = df["Image"].map(lambda x: x[0]["src"] if len(x)>0 else "" )

In [13]:
df

,Name,Price,Link,Image
0,Squier Jaguar basso (toimitus 12e),195 €,https://www.tori.fi/pirkanmaa/Squier_Jaguar_ba...,https://images.tori.fi/api/v1/imagestori/image...
1,"LP-Levyjä Pavarotti,Iglesias,Klassinen yms",1 €,https://www.tori.fi/etela-pohjanmaa/LP_Levyja_...,https://images.tori.fi/api/v1/imagestori/image...
2,Grayn anatomia - dvd,5 €,https://www.tori.fi/varsinais-suomi/Grayn_anat...,https://images.tori.fi/api/v1/imagestori/image...
3,Bassokitara,75 €,https://www.tori.fi/satakunta/Bassokitara_7765...,https://images.tori.fi/api/v1/imagestori/image...
4,Yamaha bassokitara ja Marshall vahvistin,120 €,https://www.tori.fi/lappi/Yamaha_bassokitara_j...,https://images.tori.fi/api/v1/imagestori/image...
...,...,...,...,...
1995,Nightwish Paketti,120 €,https://www.tori.fi/lappi/Nightwish_Paketti_77...,https://images.tori.fi/api/v1/imagestori/image...
1996,logitech z506 kaijutin järjestelmä,100 €,https://www.tori.fi/varsinais-suomi/logitech_z...,https://images.tori.fi/api/v1/imagestori/image...
1997,80-luvun levyt,20 €,https://www.tori.fi/etela-pohjanmaa/80_luvun_l...,https://images.tori.fi/api/v1/imagestori/image...
1998,Lasten Landola,100 €,https://www.tori.fi/kymenlaakso/Lasten_Landola...,https://images.tori.fi/api/v1/imagestori/image...


In [116]:
df2_name = df["Name"].map(lambda x: len(x) )
df2_link = df["Link"].map(lambda x: len(x))
df2_price = df["Price"].map(lambda x: int(re.search("\d{1,}",x).group()) if x != "" else 0)
#df2_price = df2_price.fillna(0)
#df2_price = df2_price.map(lambda x : print(re.))
df2_image = df["Image"].map(lambda x: int(re.search(r"\d{10}",x).group()) if re.search(r"\d{10}",x) != None else 0 )
df2_sep = df["Name"].map(lambda x: "/")
df2_value_sep = df["Name"].map(lambda x: ".")
df2 = pd.DataFrame([df2_name,df2_value_sep,df2_price,df2_value_sep,df2_link,df2_value_sep,df2_image,df2_sep])
df2 = df2.T



'https://images.tori.fi/api/v1/imagestori/images/0465334433.jpg?rule=thumb_280x210'

In [117]:
df2.to_csv("toridata.csv", sep=',')
np.savetxt(r'np.txt', df2.values, f2mt='%s')

#df.to_json("torifi.json", orient="split")    